# SREnity Agent Demo - LangGraph ReAct Implementation

This notebook demonstrates the agentic RAG system using LangGraph with a 2-node ReAct pattern:
- **Assistant Node**: Agent reasoning and tool selection
- **Tool Node**: Execute search_runbooks and search_web tools

## Features:
- Intelligent tool selection based on query analysis
- Guardrails to refuse off-topic queries
- Fallback from runbooks to web search when needed
- Clear agent reasoning visualization


## 1. Setup and Imports


In [31]:
# Install required packages
%pip install langgraph



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
# Core imports
import os
import sys
import logging
from pathlib import Path
from typing import TypedDict, Annotated, Sequence
import operator

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Set up minimal logging
logging.basicConfig(level=logging.WARNING)

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# LangChain imports
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

# Tavily search
from langchain_community.tools.tavily_search import TavilySearchResults

# Local imports
from src.utils.config import get_config, get_model_factory
from src.rag.advanced_retrieval import create_bm25_reranker_chain
from src.utils.document_loader import load_saved_documents

print("✅ All imports successful")


✅ All imports successful


## 2. Define Graph State


In [33]:
class GraphState(TypedDict):
    """State for the agent graph"""
    messages: Annotated[Sequence[BaseMessage], operator.add]

print("✅ GraphState defined")


✅ GraphState defined


## 3. Load Existing RAG Components


In [34]:
# Smart GitLab Runbook Loading with Service Filtering
from src.utils.document_loader import download_gitlab_runbooks, save_documents, load_saved_documents
from pathlib import Path

def filter_by_service(documents, services=['redis']):
    """Filter documents by service type"""
    filtered = []
    for doc in documents:
        source = doc.metadata.get('source', '').lower()
        if any(service in source for service in services):
            filtered.append(doc)
    return filtered

# Check if runbooks file exists
runbooks_file = Path("../data/runbooks/gitlab_runbooks.json")

if runbooks_file.exists():
    print("Loading saved runbooks...")
    documents = load_saved_documents()
    print(f"Loaded {len(documents)} total documents")
else:
    print("Downloading fresh runbooks...")
    documents = download_gitlab_runbooks()
    print(f"Downloaded {len(documents)} documents")
    
    print("Saving documents...")
    filepath = save_documents(documents)
    print(f"Saved to {filepath}")

# Filter to Redis services only
documents = filter_by_service(documents, ['redis'])
print(f"Filtered to {len(documents)} Redis documents")


Loading saved runbooks...
Loaded 696 total documents
Filtered to 33 Redis documents


In [16]:
# Load configuration
config = get_config()
model_factory = get_model_factory()

# Preprocess and chunk documents (same as rag_evaluation)
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
from src.utils.document_loader import preprocess_html_documents

def chunk_documents_with_tiktoken(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents using tiktoken for accurate token counting"""
    
    # Get tiktoken encoding for the configured model
    encoding = tiktoken.encoding_for_model(config.openai_model)
    
    # Create text splitter with tiktoken length function
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=lambda text: len(encoding.encode(text)),
        separators=["\n\n", "\n", " ", ""]
    )
    
    # Split documents
    chunks = text_splitter.split_documents(documents)
    
    # Calculate statistics
    total_tokens = sum(len(encoding.encode(chunk.page_content)) for chunk in chunks)
    avg_tokens = total_tokens / len(chunks) if chunks else 0
    
    print(f"Created {len(chunks)} chunks ({total_tokens:,} tokens, avg {avg_tokens:.0f} tokens/chunk)")
    
    return chunks

# Preprocess HTML documents to markdown
print("Preprocessing HTML documents to markdown...")
processed_documents = preprocess_html_documents(documents)

# Chunk the preprocessed documents
print("Chunking preprocessed documents...")
chunked_docs = chunk_documents_with_tiktoken(processed_documents, chunk_size=1000, chunk_overlap=200)

# Create BM25 + Reranker chain for runbook search
print("Creating BM25 + Reranker chain...")
bm25_reranker_chain = create_bm25_reranker_chain(chunked_docs, model_factory, bm25_k=12, rerank_k=5)
print("✅ BM25 + Reranker chain ready")

Preprocessing HTML documents to markdown...
HTML to Markdown conversion results:
  Original: 290,437 - 575,312 chars
  Markdown: 52,226 - 96,814 chars
  Reduction: 81.5%
Chunking preprocessed documents...
Created 685 chunks (631,830 tokens, avg 922 tokens/chunk)
Creating BM25 + Reranker chain...
Creating BM25 + Reranker chain...
Creating BM25 retriever from 685 documents...
BM25 retriever created (k=12)


/Users/anilyanamandra/Develop/aie8/SREnity/notebooks/../src/rag/advanced_retrieval.py:36: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank(


BM25 + Reranker chain created (BM25 k=12, Rerank k=5)
✅ BM25 + Reranker chain ready


## 4. Define Tools


In [17]:
@tool
def search_runbooks(query: str) -> str:
    """
    Search GitLab SRE runbooks for troubleshooting procedures, commands, and best practices.
    
    Use this tool for:
    - Standard SRE procedures
    - Troubleshooting steps
    - Command syntax and usage
    - Infrastructure best practices
    
    Args:
        query: The SRE question or issue to search for
    
    Returns:
        Formatted response with runbook guidance
    """
    try:
        result = bm25_reranker_chain.invoke({"question": query})
        return result["response"]
    except Exception as e:
        return f"Error searching runbooks: {str(e)}"

@tool
def search_web(query: str) -> str:
    """
    Search the web for latest updates, CVEs, version-specific issues, and recent changes.
    
    Use this tool for:
    - Recent vulnerabilities or security updates
    - Version-specific issues not in runbooks
    - Latest best practices or changes
    - Breaking changes in tools or services
    
    Args:
        query: The technical question to search for on the web
    
    Returns:
        Recent web information and updates
    """
    try:
        # Initialize Tavily search
        tavily_tool = TavilySearchResults(
            max_results=3,
            search_depth="advanced"
        )
        
        # Search with SRE context
        search_query = f"SRE DevOps {query} troubleshooting production incident"
        results = tavily_tool.invoke(search_query)
        
        # Format results
        if results:
            formatted_results = "\n\n".join([
                f"**Source:** {result.get('title', 'Unknown')}\n"
                f"**URL:** {result.get('url', 'N/A')}\n"
                f"**Content:** {result.get('content', 'No content available')}"
                for result in results
            ])
            return f"Recent web information:\n\n{formatted_results}"
        else:
            return "No recent web information found for this query."
            
    except Exception as e:
        return f"Error searching web: {str(e)}"

# Create tools list
tools = [search_runbooks, search_web]
print(f"✅ Created {len(tools)} tools:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")


✅ Created 2 tools:
  - search_runbooks: Search GitLab SRE runbooks for troubleshooting procedures, commands, and best practices.

Use this tool for:
- Standard SRE procedures
- Troubleshooting steps
- Command syntax and usage
- Infrastructure best practices

Args:
    query: The SRE question or issue to search for

Returns:
    Formatted response with runbook guidance
  - search_web: Search the web for latest updates, CVEs, version-specific issues, and recent changes.

Use this tool for:
- Recent vulnerabilities or security updates
- Version-specific issues not in runbooks
- Latest best practices or changes
- Breaking changes in tools or services

Args:
    query: The technical question to search for on the web

Returns:
    Recent web information and updates


## 5. Create LLM with Tools


In [18]:
# Create LLM with tools
llm = model_factory.get_llm()
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM configured with tools")


✅ LLM configured with tools


## 6. Define Agent Nodes


In [19]:
def assistant(state: GraphState):
    """
    Assistant node: Agent reasoning and tool selection
    """
    messages = state["messages"]
    
    # Add system message if this is the first message
    if len(messages) == 1 and isinstance(messages[0], HumanMessage):
        system_message = """
You are SREnity, an expert SRE (Site Reliability Engineer) assistant specialized in production incident response.

Your expertise includes:
- Infrastructure troubleshooting (Redis, PostgreSQL, Elastic, etc.)
- GitLab runbook procedures
- Production incident resolution
- DevOps best practices

TOOL USAGE RULES:
1. ALWAYS start with search_runbooks for SRE procedures and troubleshooting
2. Use search_web ONLY when:
   - Runbooks don't have the specific information needed
   - You need latest updates, CVEs, or version-specific issues
   - The query involves recent changes or breaking updates
3. REFUSE non-SRE queries politely but firmly

GUARDRAILS:
- If the query is clearly off-topic (weather, cooking, general knowledge, personal advice), respond:
  "I'm specialized in SRE incident response and can only help with infrastructure troubleshooting, runbook procedures, and production issues. Please ask about system operations or technical problems."
- Do NOT use tools for off-topic queries

Always provide clear, actionable guidance based on the information you find.
"""
        messages = [AIMessage(content=system_message)] + messages
    
    # Get response from LLM
    response = llm_with_tools.invoke(messages)
    
    return {"messages": [response]}

def should_continue(state: GraphState):
    """
    Conditional edge: Determine if tools need to be called
    """
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the last message has tool calls, go to tools
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    
    # Otherwise, we're done
    return END

# Create tool node
tool_node = ToolNode(tools)

print("✅ Agent nodes defined:")
print("  - assistant: Agent reasoning and tool selection")
print("  - should_continue: Conditional edge logic")
print("  - tool_node: Tool execution")


✅ Agent nodes defined:
  - assistant: Agent reasoning and tool selection
  - should_continue: Conditional edge logic
  - tool_node: Tool execution


## 7. Build and Compile Graph


In [20]:
# Build the graph
builder = StateGraph(GraphState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", tool_node)

# Set entry point
builder.add_edge(START, "assistant")

# Add conditional edge
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", END: END}
)

# Add edge from tools back to assistant
builder.add_edge("tools", "assistant")

# Compile the graph
react_graph = builder.compile()

print("✅ ReAct graph compiled successfully!")
print("\nGraph structure:")
print("START → assistant → [tools or END]")
print("              ↑          ↓")
print("              └──────────┘")


✅ ReAct graph compiled successfully!

Graph structure:
START → assistant → [tools or END]
              ↑          ↓
              └──────────┘


## 8. Test Agent with Sample Queries


In [21]:
def test_agent(query: str, verbose: bool = True):
    """
    Test the agent with a query and show the reasoning process
    """
    print(f"\n{'='*60}")
    print(f"QUERY: {query}")
    print(f"{'='*60}")
    
    # Create initial state
    initial_state = {"messages": [HumanMessage(content=query)]}
    
    # Run the graph
    if verbose:
        print("\n🤖 Agent reasoning process:")
        
    result = react_graph.invoke(initial_state, config={"recursion_limit": 10})
    
    # Extract final response
    final_message = result["messages"][-1]
    
    if verbose:
        print("\n📝 Final Response:")
        print("-" * 40)
    
    print(final_message.content)
    
    return result

print("✅ Test function ready")


✅ Test function ready


## 9. Demo Scenarios


In [22]:
# Test 1: Standard SRE query (should use runbooks only)
test_agent("How to monitor Redis memory usage?")



QUERY: How to monitor Redis memory usage?

🤖 Agent reasoning process:

📝 Final Response:
----------------------------------------
To monitor Redis memory usage effectively, you can follow these procedures:

1. Use `redis-cli` to gather memory metrics:
   - Connect to Redis: `redis-cli -h <redis_host> -p <port>`
   - Run `INFO memory` to get overall memory stats.
   - Run `MEMORY_STATS` for detailed memory allocator info.

2. Enable and use latency monitoring:
   - Set threshold: `CONFIG SET latency-monitor-threshold 100`
   - Run `LATENCY DOCTOR` for insights into latency spikes related to memory.

3. Monitor slowlog for memory-intensive commands:
   - Check slowlog entries: `SLOWLOG GET 10`
   - Review slowlog configuration: `CONFIG GET slowlog-log-slower-than` and `CONFIG GET slowlog-max-len`

4. Use Prometheus/Grafana dashboards for real-time and historical trends.

5. For advanced analysis, analyze memory dumps or keyspace patterns with specialized tools.

Key commands include `IN

{'messages': [HumanMessage(content='How to monitor Redis memory usage?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9z2ZNOFgqE2n2gLERX2EVT9C', 'function': {'arguments': '{"query":"monitor Redis memory usage"}', 'name': 'search_runbooks'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 438, 'total_tokens': 456, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_1f35c1788c', 'id': 'chatcmpl-CRp79Gw8tquLTo8YzQFBhRGFTf7gC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--23670134-e8ff-472f-9b35-7e74d334aef3-0', tool_calls=[{'name': 'search_runbooks', 'args': {'query': 'monitor Redis memory usag

In [23]:
# Test 2: Version-specific query (should use both tools)
test_agent("Redis 7.2 memory leak issues and fixes")



QUERY: Redis 7.2 memory leak issues and fixes

🤖 Agent reasoning process:


/var/folders/cz/01ttpj991sx782zbyzgb4bdc0000gn/T/ipykernel_26293/2586420026.py:43: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(



📝 Final Response:
----------------------------------------
Based on the recent web information and known issues, here are the key points regarding Redis 7.2 memory leak issues and fixes:

1. Redis 7.2 has known memory leak issues that can cause unbounded memory growth. It is crucial to ensure you are running the latest stable version of Redis 7.2 with all patches applied.

2. To troubleshoot and fix memory leaks:
   - Verify your Redis version:
     ```
     redis-server --version
     ```
   - Update Redis to the latest stable release if necessary.
   - Monitor memory usage using:
     ```
     redis-cli info memory
     ```
   - Identify large keys or memory leaks:
     ```
     redis-cli --bigkeys
     ```
   - Use profiling tools like redis-memory-analyzer for detailed analysis.
   - Clean up large or problematic keys:
     ```
     redis-cli DEL <key>
     ```
   - Adjust memory policies:
     ```
     redis-cli CONFIG SET maxmemory <bytes>
     redis-cli CONFIG SET maxmemory-pol

{'messages': [HumanMessage(content='Redis 7.2 memory leak issues and fixes', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uBzzavbld9O6JAOvHSwiEmBR', 'function': {'arguments': '{"query":"Redis 7.2 memory leak issues and fixes"}', 'name': 'search_runbooks'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 441, 'total_tokens': 465, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_1f35c1788c', 'id': 'chatcmpl-CRp7Qn0pvmqDwGUxBRx6olo3Y1A1J', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--36da1b79-f2b4-45bb-9fa4-47ee4f45ecc5-0', tool_calls=[{'name': 'search_runbooks', 'args': {'query': 'Redis 7.2

In [24]:
# Test 3: Off-topic query (should refuse)
test_agent("What's the weather like today?")



QUERY: What's the weather like today?

🤖 Agent reasoning process:

📝 Final Response:
----------------------------------------
I'm specialized in SRE incident response and can only help with infrastructure troubleshooting, runbook procedures, and production issues. Please ask about system operations or technical problems.


{'messages': [HumanMessage(content="What's the weather like today?", additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm specialized in SRE incident response and can only help with infrastructure troubleshooting, runbook procedures, and production issues. Please ask about system operations or technical problems.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 437, 'total_tokens': 470, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_1f35c1788c', 'id': 'chatcmpl-CRp7wiK6sSA9bTgHqW2CDctX5wKEx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b6887b48-8dbb-4220-b03b-9fc9fca1f08b-0', usage_metadata={'input_tokens': 437, 'output_tokens': 33, 'total_tokens

In [25]:
# Test 4: Complex SRE query (should use both tools)
test_agent("PostgreSQL connection pool exhaustion in production - how to diagnose and fix?")



QUERY: PostgreSQL connection pool exhaustion in production - how to diagnose and fix?

🤖 Agent reasoning process:

📝 Final Response:
----------------------------------------
To diagnose and fix PostgreSQL connection pool exhaustion in production, follow these steps:

1. Confirm exhaustion by checking current connection usage via your pooler dashboard or commands.
2. Diagnose the issue:
   - Check active connections with `SELECT * FROM pg_stat_activity;`.
   - Identify long-running transactions with `SELECT pid, age(current_timestamp, xact_start), query FROM pg_stat_activity WHERE ...;`.
   - Review logs for errors or slow queries.
3. Identify root causes:
   - Long transactions or idle connections.
   - High application load.
   - Misconfigured pool size.
4. Implement immediate fixes:
   - Terminate long-running or idle connections if necessary:
     ```sql
     SELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE ...;
     ```
   - Temporarily increase pool size by editing yo

{'messages': [HumanMessage(content='PostgreSQL connection pool exhaustion in production - how to diagnose and fix?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6DpZfwXl65dg2o4xbUSZVNTE', 'function': {'arguments': '{"query":"PostgreSQL connection pool exhaustion diagnosis and fix"}', 'name': 'search_runbooks'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 446, 'total_tokens': 469, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_1f35c1788c', 'id': 'chatcmpl-CRp84MRhMscvn2IR7c8bN1WsODGeG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b596d1cc-28b7-407e-94e8-a2dedeac1d61-0', tool_calls=

In [26]:
# Test 5: Command-specific query (should use runbooks)
test_agent("Show me the exact syntax for Redis MEMORY STATS command")



QUERY: Show me the exact syntax for Redis MEMORY STATS command

🤖 Agent reasoning process:

📝 Final Response:
----------------------------------------
The syntax for the Redis MEMORY STATS command is:

```
MEMORY STATS
```

You can run this command after connecting to your Redis instance using redis-cli and authenticating if necessary. It provides detailed memory usage metrics.


{'messages': [HumanMessage(content='Show me the exact syntax for Redis MEMORY STATS command', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jK7HJmFN2BRsscwu2d73Wr2V', 'function': {'arguments': '{"query":"Redis MEMORY STATS command syntax"}', 'name': 'search_runbooks'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 442, 'total_tokens': 462, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_1f35c1788c', 'id': 'chatcmpl-CRp8qrVG53zHzCjkAGy6HmX5V4bLM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f31a64f1-a8a7-40c7-b611-9c341399f3ab-0', tool_calls=[{'name': 'search_runbooks', 'args': {'query'

## 10. Interactive Demo

Run this cell to interact with the agent:


In [30]:
# Interactive demo
def interactive_demo():
    """
    Interactive demo of the SREnity agent
    """
    print("🤖 SREnity Agent Demo")
    print("Ask me about SRE procedures, troubleshooting, or production issues!")
    print("Type 'quit' to exit.\n")
    
    while True:
        query = input("You: ")
        
        if query.lower() in ['quit', 'exit', 'q']:
            print("Goodbye! 👋")
            break
            
        if query.strip():
            test_agent(query, verbose=False)
            print()

# Uncomment to run interactive demo
# interactive_demo()


## 11. Graph Visualization

Visualize the agent graph structure:


In [29]:
# Display the graph structure
try:
    from IPython.display import Image, display
    
    # Generate graph image
    graph_image = react_graph.get_graph().draw_mermaid()
    
    print("📊 Agent Graph Structure:")
    print("\nMermaid representation:")
    print(graph_image)
    
except Exception as e:
    print(f"Could not generate graph visualization: {e}")
    print("\nGraph structure:")
    print("START → assistant → [tools or END]")
    print("              ↑          ↓")
    print("              └──────────┘")


📊 Agent Graph Structure:

Mermaid representation:
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	assistant(assistant)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> assistant;
	assistant -.-> __end__;
	assistant -.-> tools;
	tools --> assistant;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

